In [1]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

In [2]:
print(tf.__version__)

2.5.0


In [3]:
import pickle
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix.pkl', 'rb') as f:    
    user_dict = pickle.load(f)
items_num=[]
for i in user_dict.values():
    items_num.append(max(i))
items_num=max(items_num)

maxlen=1000
len_seq=100

batch_size=512
epoch_num=100
hidden_size=64
keep_rate=0.9
layers_num=10
interest_num=3
neg_num=5
test_neg_num=100

In [4]:
import random
import numpy as np
def sample(user_dict,maxlen,len_seq):
    train_set=[]
    train_val_set=[]
    val_set=[]
    test_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        hist=hist[-maxlen:-2]
        #print(hist)
        for i in range(1,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #print(hist[0:i])
            seq[max(0,len_seq+idx-1):]=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            train_set.append((u,list(seq),nxt))
            #print(seq)
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        test_set.append((u,list(seq),nxt))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set,val_set,train_val_set
            
def non_zero_sample(user_dict,maxlen,len_seq):
    train_set=[]
    test_set=[]
    val_set=[]
    train_val_set=[]
    for u in user_dict.keys():
        idx=0
        hist=user_dict[u]        
        seq = np.zeros([len_seq], dtype=np.int32)
        #print(np.shape(seq[-maxlen:]))
        while(len(hist)<maxlen):
            hist.insert(0,0)
        #print(len(hist),hist)
        hist=hist[-maxlen:-2]
        for i in range(len_seq,len(hist)):
            seq = np.zeros([len_seq], dtype=np.int32)
            #rint(hist[max(0,i-len_seq):i])
            seq=hist[max(0,i-len_seq):i]
            idx+=-1
            nxt = hist[i]
            #print((u,seq,nxt))
            neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
            train_set.append((u,list(seq),nxt,neg_item))
            #print(seq)
        #print(np.shape(hist[0:len(hist)-1]))
        train_val_set.append((u,list(seq),nxt))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-2:-2]
        nxt = user_dict[u][-2]
        neg_item = [random.randint(1, items_num) for _ in range(neg_num)]
        val_set.append((u,list(seq),nxt,neg_item))
        seq = np.zeros([len_seq], dtype=np.int32)
        seq=hist[-len_seq-1:-1]
        nxt = user_dict[u][-1]
        neg_item = [random.randint(1, items_num) for _ in range(test_neg_num)]
        test_set.append((u,list(seq),nxt,neg_item))
    random.shuffle(train_set)
    random.shuffle(test_set)
    return train_set,test_set ,val_set,train_val_set   

In [5]:
'''train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)
print(len(train_set),len(test_set),len(val_set),len(train_val_set))
users_num=len(test_set)'''

'train_set,test_set,val_set,train_val_set=non_zero_sample(user_dict,maxlen,len_seq)\nprint(len(train_set),len(test_set),len(val_set),len(train_val_set))\nusers_num=len(test_set)'

In [6]:
'''users, click_seqs, pos_items,neg_items = zip(*train_set)
train_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}
users, click_seqs, pos_items,neg_items = zip(*test_set)
test_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}'''

"users, click_seqs, pos_items,neg_items = zip(*train_set)\ntrain_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}\nusers, click_seqs, pos_items,neg_items = zip(*test_set)\ntest_data = {'click_seq': np.array(click_seqs), 'pos_item': np.array(pos_items),'neg_item':np.array(neg_items)}"

In [7]:
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_train_data.pkl', 'rb') as f:
    train_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_test_data.pkl', 'rb') as f:
    test_data=pickle.load(f)
with open('/root/autodl-nas/workspace/datasets/kuairec/small_matrix_val_data.pkl', 'rb') as f:
    val_data=pickle.load(f)

In [8]:
for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(test_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(test_data['click_seq'][0],test_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
for i in range(len(train_data['click_seq'])):
    if i%500000==0:
        print(i)
    if list(val_data['click_seq'][0])==list(train_data['click_seq'][i]):
        print(val_data['click_seq'][0],val_data['pos_item'][0],train_data['click_seq'][i],train_data['pos_item'][i])
print('pass')

0
500000
0
500000
pass


In [9]:
print(np.shape(train_data['neg_item']),np.shape(test_data['neg_item']),np.shape(val_data['neg_item']))

(527766, 5) (6040, 1000) (6040, 5)


In [10]:
def scaled_dot_product_attention(q, k, v, mask):
    """Attention Mechanism Function.
    Args:
        :param q: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param k: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param v: A 3d/4d tensor with shape of (None, ..., seq_len, dim)
        :param mask: A 3d/4d tensor with shape of (None, ..., seq_len, 1)
    :return:
    """
    mat_qk = tf.matmul(q, k, transpose_b=True)  # (None, seq_len, seq_len)
    # Scaled
    dk = tf.cast(k.shape[-1], dtype=tf.float32)
    scaled_att_logits = mat_qk / tf.sqrt(dk)

    paddings = tf.ones_like(scaled_att_logits) * (-2 ** 32 + 1)  # (None, seq_len, seq_len)
    if mask!=None:
        outputs = tf.where(tf.equal(mask, tf.zeros_like(mask)), paddings, scaled_att_logits)  # (None, seq_len, seq_len)
    else:
        outputs=scaled_att_logits
    # softmax
    outputs = tf.nn.softmax(logits=outputs)  # (None, seq_len, seq_len)
    outputs = tf.matmul(outputs, v)  # (None, seq_len, dim)

    return outputs


def split_heads(x, seq_len, num_heads, depth):
    """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    Args:
        :param x: A Tensor with shape of [batch_size, seq_len, num_heads * depth]
        :param seq_len: A scalar(int).
        :param num_heads: A scalar(int).
        :param depth: A scalar(int).
    :return: A tensor with shape of [batch_size, num_heads, seq_len, depth]
    """
    x = tf.reshape(x, (-1, seq_len, num_heads, depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

In [11]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        """Multi Head Attention Mechanism.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads. If num_heads == 1, the layer is a single self-attention layer.
        :return:
        """
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        self.wq = tf.keras.layers.Dense(d_model, activation=None)
        self.wk = tf.keras.layers.Dense(d_model, activation=None)
        self.wv = tf.keras.layers.Dense(d_model, activation=None)

    def call(self, q, k, v, mask):
        q = self.wq(q)  # (None, seq_len, d_model)
        k = self.wk(k)  # (None, seq_len, d_model)
        v = self.wv(v)  # (None, seq_len, d_model)
        # split d_model into num_heads * depth
        seq_len, d_model = q.shape[1], q.shape[2]
        q = split_heads(q, seq_len, self.num_heads, q.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        k = split_heads(k, seq_len, self.num_heads, k.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        v = split_heads(v, seq_len, self.num_heads, v.shape[2] // self.num_heads)  # (None, num_heads, seq_len, depth)
        # mask
        if mask!=None:
            mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_heads, 1, 1])  # (None, num_heads, seq_len, 1)
        # attention
        scaled_attention = scaled_dot_product_attention(q, k, v, mask)  # (None, num_heads, seq_len, d_model // num_heads)
        # reshape
        outputs = tf.reshape(tf.transpose(scaled_attention, [0, 2, 1, 3]), [-1, seq_len, d_model])  # (None, seq_len, d_model)
        return outputs


class FFN(tf.keras.layers.Layer):
    def __init__(self, hidden_unit, d_model):
        """Feed Forward Network.
        Args:
            :param hidden_unit: A scalar.
            :param d_model: A scalar.
        :return:
        """
        super(FFN, self).__init__()
        self.conv1 = tf.keras.layers.Conv1D(filters=hidden_unit, kernel_size=1, activation='relu', use_bias=True)
        self.conv2 = tf.keras.layers.Conv1D(filters=d_model, kernel_size=1, activation=None, use_bias=True)

    def call(self, inputs):
        x = self.conv1(inputs)
        output = self.conv2(x)
        return output


class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads=1, ffn_hidden_unit=128, dropout=0., layer_norm_eps=1e-6):
        """Encoder Layer.
        Args:
            :param d_model: A scalar. The self-attention hidden size.
            :param num_heads: A scalar. Number of heads.
            :param ffn_hidden_unit: A scalar. Number of hidden unit in FFN
            :param dropout: A scalar. Number of dropout.
            :param layer_norm_eps: A scalar. Small float added to variance to avoid dividing by zero.
        :return:
        """
        super(TransformerEncoder, self).__init__()
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = FFN(ffn_hidden_unit, d_model)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=layer_norm_eps)

        self.dropout1 = tf.keras.layers.Dropout(dropout)
        self.dropout2 = tf.keras.layers.Dropout(dropout)

    def call(self, inputs):
        x, mask= inputs
        # self-attention
        att_out = self.mha(x, x, x, mask)  # (None, seq_len, d_model)
        att_out = self.dropout1(att_out)
        # residual add
        out1 = self.layernorm1(x + att_out)  # (None, seq_len, d_model)
        # ffn
        ffn_out = self.ffn(out1)
        ffn_out = self.dropout2(ffn_out)
        # residual add
        out2 = self.layernorm2(out1 + ffn_out)  # (None, seq_len, d_model)
        return out2
class CapsuleNetwork(tf.keras.layers.Layer):
    def __init__(self, embed_dim, seq_len, bilinear_type=0, num_interest=4, stop_grad=True):
        super(CapsuleNetwork, self).__init__()
        self.bilinear_type = bilinear_type
        self.seq_len = seq_len
        self.num_interest = num_interest
        self.embed_dim = embed_dim
        self.bi_dense=tf.keras.layers.Dense(hidden_size * self.num_interest, activation=None)
        self.stop_grad = stop_grad

    def build(self, input_shape):
        if self.bilinear_type >= 2:
            self.w = self.add_weight(
                shape=[1, self.seq_len, self.num_interest * self.embed_dim, self.embed_dim],
                initializer='random_normal',
                name='weights'
            )

    def call(self, hist_emb, mask):
        if self.bilinear_type == 0:
            hist_emb_hat = tf.tile(hist_emb, [1, 1, self.num_interest])  # (None, seq_len, num_inter * embed_dim)
        elif self.bilinear_type == 1:
            hist_emb_hat = self.bi_dense(hist_emb)
        else:
            u = tf.expand_dims(hist_emb, axis=2)  # (None, seq_len, 1, embed_dim)
            hist_emb_hat = tf.reduce_sum(self.w * u, axis=3)  # (None, seq_len, num_inter * embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.seq_len, self.num_interest, self.embed_dim])
        hist_emb_hat = tf.transpose(hist_emb_hat, [0, 2, 1, 3])  # (None, num_inter, seq_len, embed_dim)
        hist_emb_hat = tf.reshape(hist_emb_hat, [-1, self.num_interest, self.seq_len, self.embed_dim])
        if self.stop_grad:
            hist_emb_iter = tf.stop_gradient(hist_emb_hat)
        else:
            hist_emb_iter = hist_emb_hat  # (None, num_inter, seq_len, embed_dim)

        if self.bilinear_type > 0:
            '''self.capsule_weight = self.add_weight(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                initializer=tf.zeros_initializer())'''
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
            
        else:
            self.capsule_weight = tf.random.truncated_normal(
                shape=[tf.shape(hist_emb_hat)[0], self.num_interest, self.seq_len],
                stddev=1.0)
        tf.stop_gradient(self.capsule_weight)

        for i in range(3):
            att_mask = tf.tile(tf.expand_dims(mask, axis=1), [1, self.num_interest, 1])  # (None, num_inter, seq_len)
            paddings = tf.zeros_like(att_mask)

            capsule_softmax_weight = tf.nn.softmax(self.capsule_weight, axis=1)  # (None, num_inter, seq_len)
            capsule_softmax_weight = tf.where(tf.equal(att_mask, 0), paddings, capsule_softmax_weight)
            capsule_softmax_weight = tf.expand_dims(capsule_softmax_weight, 2)  # (None, num_inter, 1, seq_len)

            if i < 2:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_iter)  # (None, num_inter, 1, embed_dim)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, keepdims=True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

                delta_weight = tf.matmul(hist_emb_iter, tf.transpose(interest_capsule, [0, 1, 3, 2]))
                delta_weight = tf.reshape(delta_weight, [-1, self.num_interest, self.seq_len])
                self.capsule_weight = self.capsule_weight + delta_weight
            else:
                interest_capsule = tf.matmul(capsule_softmax_weight, hist_emb_hat)
                cap_norm = tf.reduce_sum(tf.square(interest_capsule), -1, True)
                scalar_factor = cap_norm / (1 + cap_norm) / tf.sqrt(cap_norm + 1e-9)
                interest_capsule = scalar_factor * interest_capsule

        interest_capsule = tf.reshape(interest_capsule, [-1, self.num_interest, self.embed_dim])
        return interest_capsule

In [12]:
class comirec(tf.keras.models.Model):
    def __init__(self):
        super(comirec, self).__init__()
        blocks=1
        embed_reg=0.
        num_heads=1
        layer_norm_eps=1e-6
        use_l2norm=False
        
        self.len_seq=len_seq
        self.item_embedding = tf.keras.layers.Embedding(items_num+1,hidden_size,input_length=self.len_seq,
                                                       embeddings_initializer='random_normal',
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.pos_embedding = tf.keras.layers.Embedding(len_seq,hidden_size,input_length=1,
                                                       embeddings_initializer='random_normal',
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg))
        self.dropout = tf.keras.layers.Dropout(1-keep_rate)
        
        self.capsule_network = CapsuleNetwork(hidden_size, len_seq, 1, interest_num, False)
        # norm
        self.use_l2norm = use_l2norm
        
    
    def call(self, inputs):
        seq_embed=self.item_embedding(inputs['click_seq'])
        pos_encoding = tf.expand_dims(self.pos_embedding(tf.range(self.len_seq)), axis=0)  # (1, seq_len, embed_dim)
        seq_embed += pos_encoding

        seq_embed = self.dropout(seq_embed)
        
        mask=tf.ones([tf.shape(seq_embed)[0],len_seq],dtype=tf.float32)
        #mask = tf.cast(tf.not_equal(inputs['click_seq'], 0), dtype=tf.float32)  # (None, seq_len)
        #user_hist_emb = tf.multiply(user_hist_emb, tf.expand_dims(mask, axis=-1))  # (None, seq_len, embed_dim)
        # capsule network
        user_info = self.capsule_network(seq_embed, mask)  # (b, num_inter, h) 
        
        
        
        pos_emb=self.item_embedding(tf.reshape(inputs['pos_item'], [-1, ]))
        neg_emb=self.item_embedding(inputs['neg_item'])
        
        if self.use_l2norm:
            pos_emb = tf.math.l2_normalize(pos_emb, axis=-1)
            neg_emb = tf.math.l2_normalize(neg_emb, axis=-1)
            user_info = tf.math.l2_normalize(user_info, axis=-1)
        
        
        pos_score =tf.reduce_max(tf.reduce_sum(tf.multiply(user_info, tf.expand_dims(pos_emb, axis=1)), axis=-1),axis=-1,keepdims=True)  # (None, 1)

        neg_score = tf.reduce_max(tf.reduce_sum(tf.multiply(tf.expand_dims(user_info,axis=1), tf.expand_dims(neg_emb,axis=2)), axis=-1), axis=-1)

        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
        
        
        loss = tf.reduce_mean(- tf.math.log(logits[:,0]))
        self.add_loss(loss)
        
        return logits
    
    def summary(self):
        inputs = {
            'click_seq': tf.keras.layers.Input(shape=(self.len_seq,), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'neg_item': tf.keras.layers.Input(shape=(1,), dtype=tf.int32)  # suppose neg_num=1
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [13]:
def hr(rank, k):
    """Hit Rate.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: hit rate.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1
    return res / len(rank)


def mrr(rank, k):
    """Mean Reciprocal Rank.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: mrr.
    """
    mrr = 0.0
    for r in rank:
        if r < k:
            mrr += 1 / (r + 1)
    return mrr / len(rank)


def ndcg(rank, k):
    """Normalized Discounted Cumulative Gain.
    Args:
        :param rank: A list.
        :param k: A scalar(int).
    :return: ndcg.
    """
    res = 0.0
    for r in rank:
        if r < k:
            res += 1 / np.log2(r + 2)
    return res / len(rank)


def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    for name in metric_names:
        if name == 'hr':
            res = hr(rank, k)
        elif name == 'ndcg':
            res = ndcg(rank, k)
        elif name == 'mrr':
            res = mrr(rank, k)
        else:
            break
        res_dict[name] = res
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    pred_y = - model.predict(test_data, batch_size)
    print(np.shape(pred_y))
    return eval_rank(pred_y, metric_names, k)

In [14]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
from time import time
strategy = tf.distribute.MirroredStrategy()
batch_size=batch_size*strategy.num_replicas_in_sync
with strategy.scope():
    model = comirec()
    model.summary()
    optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
    model.compile(optimizer=optimizer)
best_eval_dict={'hr':0.,'mrr':0.,'ndcg':0.}
eval_patience={'hr':0.,'mrr':0.,'ndcg':0.}
for epoch in range(1, epoch_num + 1):
    t1 = time()
    model.fit(
            x=train_data,
            epochs=1,
            batch_size=batch_size,
        validation_data=val_data)
    t2 = time()
    eval_dict = eval_pos_neg(model, val_data, ['hr', 'mrr', 'ndcg'], 1, batch_size)
    print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))
    for i in eval_dict.keys():
        if eval_dict[i]>=best_eval_dict[i]:
            best_eval_dict[i]=eval_dict[i]
        else:
            print(i,'drop 1 time')
            eval_patience[i]+=1
    if (np.array(list(eval_patience.values()))>=3).any():
        print('Trigger early stopping')
        break
eval_dict = eval_pos_neg(model, test_data, ['hr', 'mrr', 'ndcg'], 10, batch_size)
print('Iteration %d Fit [%.1f s], Evaluate [%.1f s]: HR = %.4f, MRR = %.4f, NDCG = %.4f'
              % (epoch, t2 - t1, time() - t2, eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))        

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             218688      input_1[0][0]                    
                                                                 tf.reshape[0][0]                 
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.add (TFOpLambd (None, 10, 64)       0           embedding[0][0]           

In [16]:
'''def train_model(model,ds_train,ds_valid,epoches):

    for epoch in tf.range(1,epoches+1):
        model.reset_metrics()
        
        # 在后期降低学习率
        if epoch == 5000:
            model.optimizer.lr.assign(model.optimizer.lr/2.0)
            tf.print("Lowering optimizer Learning Rate...\n\n")
        
        for x, y in ds_train:
            train_result = model.train_on_batch(x, y)

        for x, y in ds_valid:
            valid_result = model.test_on_batch(x, y,reset_metrics=False)
            
        if epoch%100 ==0:
            tf.print("epoch = ",epoch)
            print("train:",dict(zip(model.metrics_names,train_result)))
            print("valid:",dict(zip(model.metrics_names,valid_result)))
            print("")'''

'def train_model(model,ds_train,ds_valid,epoches):\n\n    for epoch in tf.range(1,epoches+1):\n        model.reset_metrics()\n        \n        # 在后期降低学习率\n        if epoch == 5000:\n            model.optimizer.lr.assign(model.optimizer.lr/2.0)\n            tf.print("Lowering optimizer Learning Rate...\n\n")\n        \n        for x, y in ds_train:\n            train_result = model.train_on_batch(x, y)\n\n        for x, y in ds_valid:\n            valid_result = model.test_on_batch(x, y,reset_metrics=False)\n            \n        if epoch%100 ==0:\n            tf.print("epoch = ",epoch)\n            print("train:",dict(zip(model.metrics_names,train_result)))\n            print("valid:",dict(zip(model.metrics_names,valid_result)))\n            print("")'

In [17]:
'''train_model(model,ds_train,ds_test,800)'''

'train_model(model,ds_train,ds_test,800)'

In [18]:
'''
items_num=3416
maxlen=100
len_seq=50
batch_size=512
epoch_num=70
hidden_size=64
keep_rate=0.9
layers_num=2
num_interest=1
neg_num=4
test_neg_num=100

567/567 [==============================] - 8s 14ms/step - loss: 0.0797
Iteration 1 Fit [8.2 s], Evaluate [0.3 s]: HR = 0.2048, MRR = 0.0723, NDCG = 0.1035
567/567 [==============================] - 9s 16ms/step - loss: 0.0793
Iteration 2 Fit [9.1 s], Evaluate [0.3 s]: HR = 0.2171, MRR = 0.0774, NDCG = 0.1102
567/567 [==============================] - 9s 15ms/step - loss: 0.0786
Iteration 3 Fit [9.0 s], Evaluate [0.2 s]: HR = 0.2142, MRR = 0.0769, NDCG = 0.1091
567/567 [==============================] - 9s 15ms/step - loss: 0.0780
Iteration 4 Fit [9.0 s], Evaluate [0.3 s]: HR = 0.2151, MRR = 0.0766, NDCG = 0.1091
567/567 [==============================] - 9s 15ms/step - loss: 0.0778
Iteration 5 Fit [8.9 s], Evaluate [0.2 s]: HR = 0.2141, MRR = 0.0776, NDCG = 0.1097
567/567 [==============================] - 9s 16ms/step - loss: 0.0770
Iteration 6 Fit [9.0 s], Evaluate [0.2 s]: HR = 0.2190, MRR = 0.0788, NDCG = 0.1117
567/567 [==============================] - 9s 15ms/step - loss: 0.0764
Iteration 7 Fit [9.0 s], Evaluate [0.2 s]: HR = 0.2151, MRR = 0.0773, NDCG = 0.1097
567/567 [==============================] - 9s 15ms/step - loss: 0.0761
Iteration 8 Fit [8.9 s], Evaluate [0.2 s]: HR = 0.2146, MRR = 0.0770, NDCG = 0.1094
567/567 [==============================] - 6s 11ms/step - loss: 0.0756
Iteration 9 Fit [6.4 s], Evaluate [0.2 s]: HR = 0.2147, MRR = 0.0775, NDCG = 0.1097
567/567 [==============================] - 8s 14ms/step - loss: 0.0751
Iteration 10 Fit [8.3 s], Evaluate [0.2 s]: HR = 0.2166, MRR = 0.0778, NDCG = 0.1104
'''

'\nitems_num=3416\nmaxlen=100\nlen_seq=50\nbatch_size=512\nepoch_num=70\nhidden_size=64\nkeep_rate=0.9\nlayers_num=2\nnum_interest=1\nneg_num=4\ntest_neg_num=100\n\n567/567 [==============================] - 8s 14ms/step - loss: 0.0797\nIteration 1 Fit [8.2 s], Evaluate [0.3 s]: HR = 0.2048, MRR = 0.0723, NDCG = 0.1035\n567/567 [==============================] - 9s 16ms/step - loss: 0.0793\nIteration 2 Fit [9.1 s], Evaluate [0.3 s]: HR = 0.2171, MRR = 0.0774, NDCG = 0.1102\n567/567 [==============================] - 9s 15ms/step - loss: 0.0786\nIteration 3 Fit [9.0 s], Evaluate [0.2 s]: HR = 0.2142, MRR = 0.0769, NDCG = 0.1091\n567/567 [==============================] - 9s 15ms/step - loss: 0.0780\nIteration 4 Fit [9.0 s], Evaluate [0.3 s]: HR = 0.2151, MRR = 0.0766, NDCG = 0.1091\n567/567 [==============================] - 9s 15ms/step - loss: 0.0778\nIteration 5 Fit [8.9 s], Evaluate [0.2 s]: HR = 0.2141, MRR = 0.0776, NDCG = 0.1097\n567/567 [==============================] - 9s 16ms

In [19]:
!nvidia-smi

Wed Feb  1 15:33:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:47:00.0 Off |                  N/A |
| 50%   44C    P2   103W / 320W |   8749MiB / 10240MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------